In [1]:
import os
import re
import lpips
import torch
from PIL import Image
import torchvision.transforms as T

# 1) Init LPIPS model
loss_fn = lpips.LPIPS(net='alex').cuda()  
loss_fn.eval()

# 2) Pre‐processing to [-1,1]
transform = T.Compose([
    T.Resize((512, 512)),            
    T.ToTensor(),
    T.Normalize((0.5,)*3, (0.5,)*3),  
])


def load_tensor(path):
    img = Image.open(path).convert('RGB')
    t = transform(img).unsqueeze(0)
    return t.cuda() if torch.cuda.is_available() else t

def find_file_with_any_ext(folder, base_name):
    for ext in ["jpg", "jpeg", "png", "bmp"]:
        path = os.path.join(folder, f"{base_name}.{ext}")
        if os.path.isfile(path):
            return path
    return None

def average_lpips_from_folder(stylized_folder, content_folder, style_folder):
    pattern = re.compile(r'(.+)_stylized_(.+)\.(png|jpg|jpeg|bmp)$', re.IGNORECASE)
    scores_stylepreservation = []
    scores_contentpreservation = []

    for fname in sorted(os.listdir(stylized_folder)):
        m = pattern.match(fname)
        if not m:
            continue
        content_name, style_name, ext = m.groups()
        stylized_path = os.path.join(stylized_folder, fname)
        content_path = find_file_with_any_ext(content_folder, content_name)
        style_path = find_file_with_any_ext(style_folder, style_name)
        
        if not content_path:
            print(f"  → Skipping {fname}: no content file found")
            continue
        if not style_path:
            print(f"  → Skipping {fname}: no style file found")
            continue

        stylized = load_tensor(stylized_path)
        content = load_tensor(content_path)
        style = load_tensor(style_path)

        with torch.no_grad():
            score_style = loss_fn(stylized, style).item()
            score_content = loss_fn(stylized, content).item()

        scores_stylepreservation.append(score_style)
        scores_contentpreservation.append(score_content)

    if not scores_stylepreservation:
        print("No valid stylized/content/style triples found in folder.")
        return

    print("\nSTYLE PRESERVATION")
    avg = sum(scores_stylepreservation) / len(scores_stylepreservation)
    print(f"Computed LPIPS on {len(scores_stylepreservation)} pairs.  Average LPIPS = {avg:.4f}")

    print("\nCONTENT PRESERVATION")
    avg = sum(scores_contentpreservation) / len(scores_contentpreservation)
    print(f"Computed LPIPS on {len(scores_contentpreservation)} pairs.  Average LPIPS = {avg:.4f}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/opt/anaconda-2024.02/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda-2024.02/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/cv27f25/.local/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth


In [3]:
if __name__ == "__main__":
    stylized_folder = "../StyTR-2/out1"
    content_folder = "resized_content_test_small"
    style_folder = "../data/style_test_small"
    average_lpips_from_folder(stylized_folder, content_folder, style_folder)



STYLE PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.6760

CONTENT PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.5286


In [4]:
if __name__ == "__main__":
    stylized_folder = "../StyTR-2/out"
    content_folder = "../StyTR-2/input/content"
    style_folder = "../StyTR-2/input/style"
    average_lpips_from_folder(stylized_folder, content_folder, style_folder)


STYLE PRESERVATION
Computed LPIPS on 48 pairs.  Average LPIPS = 0.7044

CONTENT PRESERVATION
Computed LPIPS on 48 pairs.  Average LPIPS = 0.5948


In [ ]:
if __name__ == "__main__":
    stylized_folder = "../StyTR-2/out_selftrained"
    content_folder = "resized_content_test_small"
    style_folder = "../data/style_test_small"
    average_lpips_from_folder(stylized_folder, content_folder, style_folder)
